In [2]:
import os
import json
import glob
import pandas as pd
import sqlglot
from sqlglot import exp, parse_one

# Configuration
SQL_DIR = "./spider2-snow/evaluation_suite/gold/sql"
DB_ROOT = "./spider2-snow/resource/databases"

def get_table_meta(catalog, schema, table):
    """Loads metadata from the corresponding JSON file."""
    # Handle missing catalog/schema by searching if necessary
    if not catalog:
        # Search all DBs for the table
        for db_name in os.listdir(DB_ROOT):
            db_path = os.path.join(DB_ROOT, db_name)
            if not os.path.isdir(db_path): continue
            for sch_name in os.listdir(db_path):
                sch_path = os.path.join(db_path, sch_name)
                if not os.path.isdir(sch_path): continue
                meta_path = os.path.join(sch_path, f"{table.upper()}.json")
                if os.path.exists(meta_path):
                    with open(meta_path, 'r') as f: return json.load(f)
    else:
        path = os.path.join(DB_ROOT, catalog.upper(), schema.upper() if schema else "", f"{table.upper()}.json")
        if os.path.exists(path):
            with open(path, 'r') as f: return json.load(f)
    return None

def analyze_sql(file_path):
    with open(file_path, 'r') as f:
        sql = f.read()
    
    try:
        # Snowflake dialect is common in Spider2-Snow
        parsed = parse_one(sql, read="snowflake")
    except Exception as e:
        return f"Error parsing {os.path.basename(file_path)}: {e}"

    results = []
    tables = parsed.find_all(exp.Table)
    all_columns = {c.name.upper() for c in parsed.find_all(exp.Column)}

    for t in tables:
        catalog = t.catalog.upper() if t.catalog else None
        schema = t.db.upper() if t.db else None
        table_name = t.this.name.upper()
        
        meta = get_table_meta(catalog, schema, table_name)
        if meta:
            db_info = {
                "database": catalog or meta.get("table_fullname", "").split('.')[0],
                "table": f"{schema}.{table_name}" if schema else table_name,
                "columns": {}
            }
            
            meta_cols = [c.upper() for c in meta.get("column_names", [])]
            meta_types = meta.get("column_types", [])
            sample_rows = meta.get("sample_rows", [])
            
            for i, col in enumerate(meta_cols):
                if col in all_columns:
                    db_info["columns"][col] = {
                        "type": meta_types[i] if i < len(meta_types) else "UNKNOWN",
                        "examples": [row.get(col) for row in sample_rows[:5]]
                    }
            results.append(db_info)
            
    return results

In [ ]:
sql_files = sorted(glob.glob(os.path.join(SQL_DIR, "*.sql")))
all_analyses = {}

print(f"Analyzing {len(sql_files[:10])} SQL files...")
for f in sql_files[:10]: # Process first 10 for demonstration
    fname = os.path.basename(f)
    all_analyses[fname] = analyze_sql(f)

# Output results as formatted JSON
print(json.dumps(all_analyses, indent=2))

# Flatten for DataFrame view
flattened = []
for file, analysis in all_analyses.items():
    if isinstance(analysis, list):
        for table_data in analysis:
            for col, col_data in table_data['columns'].items():
                flattened.append({
                    "SQL": file,
                    "DB": table_data['database'],
                    "Table": table_data['table'],
                    "Column": col,
                    "Type": col_data['type'],
                    "Examples": col_data['examples']
                })
